In [21]:
import logging
from os import environ
logging.disable(logging.WARNING)  # Suppress deprecation warnings

In [22]:
import numpy as np
import time
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
from keras.preprocessing.image import ImageDataGenerator

In [23]:
#Model params(
img_rows   = 80  # Training data size is 480
img_cols   = 80  # Training data size is 480
rescale    = 1./255
batch_size = 10
num_epoch  = 10
classes    = ['d4', 'd6', 'd8', 'd10', 'd12', 'd20']
directory  = './dice/'

In [35]:
print('Data preprocessing')
input_shape = (img_rows, img_cols, 1)
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    directory   = directory + './train/',
    target_size = (img_rows, img_cols),
    classes     = classes,
    color_mode  = 'grayscale',
    batch_size  = batch_size,
)
        
valid_generator = ImageDataGenerator(rescale=1./255,).flow_from_directory(
    directory   = directory + './valid/',
    target_size = (img_rows, img_cols),
    classes     = classes,
    color_mode  = 'grayscale',
    batch_size  = batch_size,
)

Data preprocessing
Found 14284 images belonging to 6 classes.
Found 2102 images belonging to 6 classes.


In [40]:
# Model design
model = Sequential()

# Adding convolution and pooling layers
for x in range(2):
    model.add(Conv2D(8,kernel_size=(3,3),activation='relu',input_shape=input_shape))
    model.add(Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=input_shape))
    model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape))
    model.add(Conv2D(64,kernel_size=(3,3),activation='relu',input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

# Adding fully connected layer
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))

# Adding softmax layer for final categorization
model.add(Dense(len(classes),activation='softmax'))

# Compile model with Adadelta optimizer
model.compile(loss=categorical_crossentropy,optimizer=Adadelta(),metrics=['accuracy'])

# Model summary:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_112 (Conv2D)          (None, 78, 78, 8)         80        
_________________________________________________________________
conv2d_113 (Conv2D)          (None, 76, 76, 16)        1168      
_________________________________________________________________
conv2d_114 (Conv2D)          (None, 74, 74, 32)        4640      
_________________________________________________________________
conv2d_115 (Conv2D)          (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_35 (Dropout)         (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_116 (Conv2D)          (None, 34, 34, 8)         4616      
__________

In [42]:
# Model training
start = time.time()

model_hist = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epoch,
    verbose=1,
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)
        
end = time.time()
print("Training time: {:.0f}s".format(end-start))

Epoch 1/10
1429/1429 [==============================] - 171s 120ms/step - loss: 1.2736 - acc: 0.4941 - val_loss: 0.9826 - val_acc: 0.6746
Epoch 2/10
1429/1429 [==============================] - 175s 123ms/step - loss: 0.5777 - acc: 0.7934 - val_loss: 0.3440 - val_acc: 0.8963
Epoch 3/10
1429/1429 [==============================] - 166s 116ms/step - loss: 0.3220 - acc: 0.8911 - val_loss: 0.4215 - val_acc: 0.8968
Epoch 4/10
1429/1429 [==============================] - 159s 111ms/step - loss: 0.2084 - acc: 0.9339 - val_loss: 0.2768 - val_acc: 0.9515
Epoch 5/10
1429/1429 [==============================] - 161s 113ms/step - loss: 0.1567 - acc: 0.9508 - val_loss: 0.2599 - val_acc: 0.9577
Epoch 6/10
1429/1429 [==============================] - 148s 104ms/step - loss: 0.1426 - acc: 0.9558 - val_loss: 0.3222 - val_acc: 0.9343
Epoch 7/10
1429/1429 [==============================] - 151s 105ms/step - loss: 0.1234 - acc: 0.9617 - val_loss: 0.2491 - val_acc: 0.9724
Epoch 8/10
1429/1429 [============

In [48]:
# Model results on the ./test dataset:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    directory   ='./test/',
    target_size =(img_rows, img_cols),
    classes     =classes,
    color_mode  ='grayscale',
    batch_size  =batch_size
)
    
model_results = model.evaluate_generator(test_generator, steps=len(test_generator))
model_results

for idx, label in enumerate(model.metrics_names):
    print('{}: {}'.format(label, model_results[idx]))

Found 40 images belonging to 6 classes.
loss: 0.00357031896806
acc: 1.0
